In [40]:
import feedparser
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv
import os

#APIキーを読み込む
load_dotenv(".env")
api_key = os.getenv("API_KEY")
client = OpenAI(api_key=api_key)


In [41]:
#デザイン系ブログのRSSから記事を取る
FEED_URL = "https://coliss.com/feed"

feed = feedparser.parse(FEED_URL)


In [42]:
#記事の本文を取る関数
def fetch_article_text(url):
    res = requests.get(url, headers={"User-Agent": "MyScraper/1.0"})
    soup = BeautifulSoup(res.text, "html.parser")
    # とりあえず<p>タグのテキストを全部まとめてみる
    paragraphs = [p.get_text() for p in soup.find_all("p")]
    text = "\n".join(paragraphs)
    return text


In [43]:
#LLMで要約する関数
def summarize_with_llm(title, content):
    prompt = f"""
あなたは記事を要約するアシスタントです。
次の記事を読んで、
1) 要点を3つ（箇条書き）
2) 学べることを1〜2文
にまとめてください。

【記事タイトル】
{title}

【本文】
{content[:1000]}  #先頭だけ取る
"""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
    return resp.choices[0].message.content


In [44]:
# 最新のfeedを再取得
r = requests.get(FEED_URL, timeout=10, headers={"User-Agent": "MyScraper/1.0"})
feed = feedparser.parse(r.text)

N = 2

for entry in feed.entries[:3]:  #上位3件を取得する
    title = getattr(entry, "title", "(no title)")
    url   = getattr(entry, "link", "")

    print("記事タイトル:", title)
    print("URL:", url)

    content = fetch_article_text(url) if url else ""
    if not content.strip():
        summary_text = BeautifulSoup(getattr(entry, "summary", ""), "html.parser").get_text()
        content = summary_text or "(本文なし)"

    summary = summarize_with_llm(title, content)
    print("\n--- 要約結果 ---")
    print(summary)

else:
    print("記事が見つかりませんでした。")


記事タイトル: これは助かる！ AIエディタ Cursorから公式の日本語ドキュメントがローンチ、要注目のポイントを解説
URL: https://coliss.com/articles/build-websites/operation/work/cursor-official-docs.html

--- 要約結果 ---
1) 要点
- AIエディタCursorの公式日本語ドキュメントがローンチされ、12言語に対応している。
- 検索機能が高速化され、ユーザーインターフェースが改善されている。
- Model Context Protocol (MCP)を利用することで、外部ツールやデータソースとの接続が容易になった。

2) 学べること
Cursorの公式ドキュメントの日本語版が登場したことで、より多くのユーザーが使いやすくなり、効率的に情報を得られるようになった。特に、検索機能やインターフェースの改善は、ユーザー体験を向上させる重要な要素である。
記事タイトル: セール期間は明日まで！ イラスト、同人誌、ゲームなど、創作に役立つ資料や辞典がKindle本 読書の秋セールでお買い得です
URL: https://coliss.com/articles/products/kindle-sales-20250924.html

--- 要約結果 ---
1) 要点
- Kindle本の「読書の秋セール」で、イラストや同人誌、ゲーム制作に役立つ資料や辞典が最大70％OFFで販売中。
- セールは明日9月25日までで、対象の本はファンタジー衣装や歴史・文化、SFに関する事典が含まれている。
- これらの資料は、シナリオ作りやキャラクターデザインに役立つ情報を辞典形式で提供している。

2) 学べること
このセールを利用することで、創作活動に必要な知識や資料を手に入れ、作品のクオリティを向上させることができる。特に、ファンタジーやSFの設定を深めるための参考資料が豊富に揃っている。
記事タイトル: Kindle本 読書の秋セールが開催！ セール初登場もたくさんあり、Web制作やUIデザイン、イラスト書がかなりお買い得です
URL: https://coliss.com/articles/products/kindle-sales-20250922.html

--- 要